In [17]:
import torch
import cv2
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
def test_image_show(image_path):
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    plt.imshow(image)
    plt.axis('off')
    plt.show()

In [3]:
# def show_predictions(predictions, image_path, iamge_name):
#     # Load the image
#     image = cv2.imread(image_path)
#     image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

#     # Create a figure and axis
#     fig, ax = plt.subplots(1, figsize=(12, 8))
#     ax.imshow(image)

#     # Plot each prediction
#     for pred in predictions:
#         label, score, bbox = pred  # unpack tuple

#         # Draw bounding box
#         rect = plt.Rectangle((bbox[0], bbox[1]), bbox[2] - bbox[0], bbox[3] - bbox[1],
#                              linewidth=2, edgecolor='red', facecolor='none')
#         ax.add_patch(rect)

#         # Add label and score
#         ax.text(bbox[0], bbox[1] - 10, f'{label}: {score:.2f}', color='red', fontsize=12, 
#                 bbox=dict(facecolor='white', alpha=0.5, edgecolor='none'))

#     plt.axis('off')
#     plt.show()
#     fig.savefig(os.path.join('/ExDark-Object-Detection/output', image_name))
#     plt.close(fig)


In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

def detect_image_yolov5(image_path, model_path='/ExDark-Object-Detection/models/YOLOv5.pt'):
    # Load YOLOv5 model
    model = torch.hub.load('ultralytics/yolov5', 'custom', path=model_path, device=device)
    model.eval()

    # Đọc ảnh bằng OpenCV
    img = cv2.imread(image_path)
    if img is None:
        raise ValueError(f"Không thể đọc ảnh từ đường dẫn: {image_path}")

    # Convert BGR -> RGB vì OpenCV dùng BGR, YOLOv5 dùng RGB
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    # Dự đoán
    results = model(img_rgb)

    # Kết quả dạng pandas DataFrame
    df = results.pandas().xyxy[0]  # bounding boxes theo dạng (x1, y1, x2, y2)

    output = []
    for _, row in df.iterrows():
        label = row['name']
        conf = float(row['confidence'])
        bbox = (int(row['xmin']), int(row['ymin']), int(row['xmax']), int(row['ymax']))
        output.append((label, conf, bbox))
    
    return output


In [ ]:
# Use the default YOLOv5s model if custom model is not available
output = detect_image_yolov5('/ExDark-Object-Detection/input/.jpg')
print(output) # still dunno why i cant print the prediction plot :))))

Using cache found in C:\Users\DELL Pre 7670/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2025-6-4 Python-3.12.4 torch-2.6.0+cu124 CUDA:0 (NVIDIA RTX A1000 Laptop GPU, 4096MiB)

Fusing layers... 
custom_YOLOv5s summary: 232 layers, 7257306 parameters, 0 gradients
Adding AutoShape... 


[]


C:\Users\DELL Pre 7670/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
